# Convolutional Neural Network

### Importing the libraries

In [ ]:
import pandas as pd
import tensorflow as tf
from tensorflow import keras
from keras import layers
import numpy as np
from keras.preprocessing.image import ImageDataGenerator

In [ ]:
tf.__version__

## Part 1 - Data Preprocessing

###Importing the data

In [ ]:
classes = ["unknown", "head lamp", "door scratch", "glass shatter", "tail lamp", "bumper dent", "bumper scratch", "door dent"]
print(sorted(classes))
directory = 'car_wreck_csv/'
df = pd.read_csv(directory+'data.csv')
df.drop(df.index[1512:], inplace=True)

### Preprocessing the Training set

In [ ]:
train_datagen = ImageDataGenerator(rescale = 1./224,
                                   shear_range = 0.2,
                                   zoom_range = 0.2,
                                   horizontal_flip = True,
                                   validation_split=0.1)
train_set = train_datagen.flow_from_dataframe( df,
                                        directory=directory,
                                        x_col="image",
                                        y_col="classes",
                                        target_size=(224,224),
                                        batch_size=8,
                                        class_mode="categorical",
                                        subset="training"
)

### Preprocessing the validation set

In [ ]:
val_datagen = ImageDataGenerator(rescale = 1./224, validation_split=0.1)
val_set = val_datagen.flow_from_dataframe(df,
                                  directory=directory,
                                  x_col="image",
                                  y_col="classes",
                                  target_size=(224,224),
                                  batch_size=8,
                                  class_mode="categorical",
                                  subset="validation"
)

## Part 2 - Building the CNN

The commented out blocks of the code are all addiction added to try to improve the val_accuracy of this model.

In [ ]:
model = keras.Sequential([
    layers.Conv2D(filters=32, kernel_size=3, activation='relu', input_shape=[224, 224, 3]),
    layers.MaxPool2D(pool_size=2, strides=2),
    #layers.Dropout(0.5),
    layers.Conv2D(filters=32, kernel_size=3, activation='relu'),
    layers.MaxPool2D(pool_size=2, strides=2),
    #layers.Dropout(0.5),
    #layers.Conv2D(filters=32, kernel_size=3, activation='relu'),
    #layers.MaxPool2D(pool_size=2, strides=2),
    #layers.Conv2D(filters=64, kernel_size=3, activation='relu'),
    #layers.MaxPool2D(pool_size=2, strides=2),
    layers.Flatten(),
    #layers.BatchNormalization(),
    #layers.Dropout(0.5),
    layers.Dense(units=64, activation="relu"),
    #layers.BatchNormalization(),
    layers.Dense(units=8, activation = "softmax")
])

## Part 3 - Training the CNN

### Compiling the CNN

In [ ]:
model.compile(
    optimizer=keras.optimizers.Adam(0.001),
    loss='categorical_crossentropy',
    metrics=["accuracy"]
)

### Training the CNN on the Training set and evaluating it on the Test set

In [ ]:
model.fit(
    train_set,
    validation_data=val_set,
    epochs=20
    #callbacks=[tf.keras.callbacks.EarlyStopping(
        #monitor='val_accuracy',
        #min_delta=0.01,
        #patience=4,
        #mode='auto',
        #baseline=0.30,
        #restore_best_weights=True
    #)]
)

## Part 4 - Making a single prediction

In [ ]:
test_image = tf.keras.utils.load_img(directory + 'test1.jpg', target_size=(224, 224))
test_image = tf.keras.utils.img_to_array(test_image)
test_image = np.expand_dims(test_image, axis=0)
print(model.predict(test_image))
result = np.argmax(model.predict(test_image))
predicted_label = sorted(classes)[result]



In [ ]:
print(predicted_label)

Having trained the model and used it to predict with different variations of this code's hyper-parameters, we notice that the val accuracy doesn't go higher than around 57%. 

This might be the consequence of having a relatively small dataset of only 1500 images and 8 classes. 

A suggested solution is to use transfered learning with a pre trained base in car features.